In [1]:
%matplotlib inline

In [2]:
import lib
import interp
import nc
import numpy as np


In [3]:
# the input data "winGlink_3dmod_input.xyzv" had no information about the crs.
# so, we had to hard code it.
# epsg represents the data area are in WGS84/ UTM 53S

source_crs = lib.epsg_to_crs(32753)

# make a grid that covers the whole area of all the points
# with desired resolution. The grid crs we want is 4326.
geo_grid = lib.Grid3D.from_extent_and_resolution(left=(131.5, -31., -4500.),
                                                 right=(132.5, -30., -125.),
                                                 resolution=(0.005, 0.005, 100.),
                                                 crs=lib.wgs84_crs)
# utm points are poivins flattened in our data crs
utm_points = geo_grid.flatten().to_crs(source_crs)

utm_grid = utm_points.to_gridded_array(geo_grid.shape)

In [4]:
utm_points.points.shape

(1760000, 3)

In [5]:
utm_grid.shape

(200, 200, 44, 3)

In [6]:
# the input data are in x y z v order and is fixed for false origin
arr = np.loadtxt('winGlink_3dmod_input.xyzv')

# source_points also remembers its crs
source_points = lib.Points3D(arr[:, 0:3], source_crs).fix_false_origin()

In [7]:
source_val = arr[:, 3]
#print(source_val.shape)

source_in_geo = source_points.to_crs(lib.wgs84_crs)

mask = source_in_geo.clipping_mask(geo_grid)
#print(mask.shape)
source_points.points = source_points.points[mask]

source_val = source_val[mask]

In [8]:
#resistivity = interp.nearest(source_points, source_val, utm_points)
resistivity = interp.IDW(source_points, source_val, utm_points)

In [9]:
resistivity = resistivity.reshape(geo_grid.shape[:3])

print(resistivity.shape)

(200, 200, 44)


In [10]:
nc.write_resistivity_grid('IDW.nc', lib.wgs84_crs, geo_grid.y, geo_grid.x, geo_grid.z, resistivity.transpose([2, 0, 1]))

